In [ ]:
import torch
import numpy as np
import os
from EduNLP.Pretrain import DisenQTokenizer, train_disenQNet
from EduNLP.Vector import DisenQModel, T2V
from EduNLP.I2V import DisenQ
from EduNLP.ModelZoo import load_items

os.environ["CUDA_VISIBLE_DEVICES"]= "0,1"
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")

# 训练自己的disenQNet模型
## 1. 数据

In [ ]:
data_dir =  "../../tests/test_vec"
output_dir ="../test_modeldisenq/disenq_s"

## 2. 训练和评估

In [ ]:
train_params = {
    'epoch': 50,
    'batch': 256,
    'trim_min': 5,
    'max_len': 250,
    "hidden": 128,
    'warm_up': 5,
    "device": "cuda"
}

disen_data_train = load_items(f"{data_dir}/disenq_train.json")
disen_data_test = load_items(f"{data_dir}/disenq_test.json")

train_disenQNet(
    disen_data_train,
    output_dir,
    output_dir,
    train_params=train_params,
    test_items=disen_data_test,
)

## 3.使用

In [ ]:
vocab_path = os.path.join(output_dir, "vocab.list")
tokenizer_kwargs = {
    "vocab_path": vocab_path,
    "max_length": 250,
    "tokenize_method": "space",
}
i2v = DisenQ('disenQ', 'disenq', output_dir, tokenizer_kwargs=tokenizer_kwargs, device="cuda")

test_items = [
    {"content": "10 米 的 (2/5) = 多少 米 的 (1/2),有 公 式"},
    {"content": "10 米 的 (2/5) = 多少 米 的 (1/2),有 公 式 , 如 图 , 若 $x,y$ 满 足 约 束 条 件 公 式"},
]

t_vec = i2v.infer_token_vector(test_items[1], key=lambda x: x["content"])
i_vec = i2v.infer_item_vector(test_items[1], key=lambda x: x["content"], vector_type="k")
print(i_vec.shape) # == torch.Size([1, 128])
print(t_vec.shape) # == torch.Size([1, 150, 128])